# Importing Dataset from Google Drive

In [ ]:
#!pip install datasets transformers torchaudio

In [1]:
! pip install audiomentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 2.5 MB/s eta 0:00:00


In [2]:
!pip install gdown

  Using cached gdown-5.2.0-py3-none-any.whl.metadata (5.8 kB)
Using cached gdown-5.2.0-py3-none-any.whl (18 kB)


In [3]:
import gdown
# Replace with your own file ID
file_id = '1HUK5kT54xWiy41bqQTiIe-fF8sOj7yvS'
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'final_output_vanilla_EYASE_EAED.zip', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1HUK5kT54xWiy41bqQTiIe-fF8sOj7yvS
From (redirected): https://drive.google.com/uc?id=1HUK5kT54xWiy41bqQTiIe-fF8sOj7yvS&confirm=t&uuid=d7e3eecb-40e5-45ff-8fc4-f17f73212901
To: /kaggle/working/final_output_vanilla_EYASE_EAED.zip
100%|██████████| 662M/662M [00:16<00:00, 40.9MB/s] 


'final_output_vanilla_EYASE_EAED.zip'

In [4]:
!unzip -q final_output_vanilla_EYASE_EAED.zip 

# Importing Libraries

In [7]:
import os
import torch
import librosa
import torchaudio
import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sns
import soundfile as sf
import matplotlib.pyplot as plt
import torch.nn.functional as F
from datasets import Dataset,load_metric
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
#from audiomentations import Compose, TimeStretch, PitchShift, AddBackgroundNoise
from transformers import AutoProcessor, Wav2Vec2Processor, Wav2Vec2ForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TrainerCallback



2024-06-30 11:18:42.898656: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 11:18:42.898763: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 11:18:43.026995: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Spliting data set to train and test

In [11]:
data_dir = 'final_output_vanilla_EYASE_EAED'

# List all files in the directory
files = os.listdir(data_dir)
print(f'Total files: {len(files)}')
print(files[:10])  # Print the first 10 files as a sample

Total files: 3541
['FEM_EnasKamel_ang_2075.wav', 'MAL_AsserYassin_neu_2682.wav', 'MAL_KhaledSarhan_sad_1640.wav', 'FEM_Zeina_sad_856.wav', 'FEM__EYASE_ang_100016.wav', 'MAL_YoussefSherief_neu_2466.wav', 'MAL__EYASE_neu_100418.wav', 'FEM_Lebleba_ang_1446.wav', 'FEM_NadiaKhairy_sad_1529.wav', 'MAL__EYASE_neu_100323.wav']


In [12]:
# List all audio files in the directory
audio_files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.wav')]
emotions = [file.split('_')[-2] for file in audio_files]

# Splitting into train (70%), validation (15%), and test (15%)
train_files, test_files, train_emotions, test_emotions = train_test_split(audio_files, emotions, test_size=0.3, random_state=42, stratify=emotions)
val_files, test_files, val_emotions, test_emotions = train_test_split(test_files, test_emotions, test_size=0.5, random_state=42, stratify=test_emotions)

print(f'Training files: {len(train_files)}')
print(f'Validation files: {len(val_files)}')
print(f'Test files: {len(test_files)}')

# Convert the list to a pandas Series for easy counting
emotions_series = pd.Series(emotions)

# Count occurrences of each emotion
emotion_counts = emotions_series.value_counts()

# Display the counts
print("Emotion Counts:")
print(emotion_counts)

print(f"Train emotions distribution: {dict(zip(*np.unique(train_emotions, return_counts=True)))}")
print(f"Validation emotions distribution: {dict(zip(*np.unique(val_emotions, return_counts=True)))}")
print(f"Test emotions distribution: {dict(zip(*np.unique(test_emotions, return_counts=True)))}")

Training files: 2478
Validation files: 531
Test files: 532
Emotion Counts:
neu    1154
ang    1059
sad     768
hap     560
Name: count, dtype: int64
Train emotions distribution: {'ang': 741, 'hap': 392, 'neu': 808, 'sad': 537}
Validation emotions distribution: {'ang': 159, 'hap': 84, 'neu': 173, 'sad': 115}
Test emotions distribution: {'ang': 159, 'hap': 84, 'neu': 173, 'sad': 116}


In [ ]:
'''# Convert the list to a pandas Series for easy counting
train_emotions_series = pd.Series(train_emotions)

# Count occurrences of each emotion
train_emotion_counts = train_emotions_series.value_counts()
# Create directories for augmented files
train_aug_dir = os.path.join(data_dir, 'train_augmented')
os.makedirs(train_aug_dir, exist_ok=True)

# Augmentation function
def augment_audio(input_file, output_file, time_stretch_factor=0.2, pitch_shift_steps=1, noise_path='path_to_noise_files'):
    augment = Compose([
        TimeStretch(min_rate=0.8, max_rate=1.2),
        PitchShift(min_semitones=-pitch_shift_steps, max_semitones=pitch_shift_steps),
        AddBackgroundNoise(sounds_path=noise_path)
    ])

    # Load audio file and apply augmentation
    audio, sample_rate = librosa.load(input_file, sr=None)
    augmented_audio = augment(samples=audio, sample_rate=sample_rate)

    # Save augmented audio
    sf.write(output_file, augmented_audio, sample_rate)

# Augmentation function
def augment_audio(input_file, output_file, time_stretch_factor=0.2, pitch_shift_steps=1, noise_std=0.005):
    augment = Compose([
        TimeStretch(min_rate=0.8, max_rate=1.2),
        PitchShift(min_semitones=-pitch_shift_steps, max_semitones=pitch_shift_steps)
    ])

    # Load audio file and apply augmentation
    audio, sample_rate = librosa.load(input_file, sr=None)
    augmented_audio = augment(samples=audio, sample_rate=sample_rate)

    # Add Gaussian noise
    noise = np.random.normal(0, noise_std, augmented_audio.shape)
    augmented_audio = augmented_audio + noise

    # Save augmented audio
    sf.write(output_file, augmented_audio, sample_rate)

# Function to augment a specific class
def augment_class(files, target_count, augment_dir):
    count = len(files)
    augmented_emotions = []
    for i in range(target_count - count):
        file = files[i % count]
        file_name = os.path.basename(file)
        aug_file_name = f"aug_{i}_{file_name}"
        aug_file_path = os.path.join(augment_dir, aug_file_name)
        augment_audio(file, aug_file_path)
        # Append the corresponding emotion to augmented_emotions
        augmented_emotions.append(train_emotions[train_files.index(file)])
    
    return augmented_emotions

# Separate files by emotion
hap_files = [file for file, emotion in zip(train_files, train_emotions) if emotion == 'hap']
neu_files = [file for file, emotion in zip(train_files, train_emotions) if emotion == 'neu']
sad_files = [file for file, emotion in zip(train_files, train_emotions) if emotion == 'sad']

# Determine the target count for augmentation (using 'ang' as a reference)
target_count = train_emotion_counts['ang']
print(target_count)

# Augment "hap" and "neu" classes to balance them
hap_augmented_emotions = augment_class(hap_files, target_count, train_aug_dir)
neu_augmented_emotions = augment_class(neu_files, target_count, train_aug_dir)
sad_augmented_emotions = augment_class(sad_files, target_count, train_aug_dir)

# Append augmented files and their emotions to the training set
train_files += hap_files
train_files += neu_files
train_files += sad_files

train_emotions += hap_augmented_emotions
train_emotions += neu_augmented_emotions
train_emotions += sad_augmented_emotions

# Convert the list to a pandas Series for easy counting
train_emotions_series = pd.Series(train_emotions)

# Count occurrences of each emotion in the augmented train set
augmented_emotion_counts = train_emotions_series.value_counts()

# Display the counts
print("Augmented Emotion Counts in Train Set:")
print(augmented_emotion_counts)'''

# Define Helper Functions

In [13]:
# Function to load and preprocess the audio files
def load_and_preprocess(file_path, target_sr=16000):
    try:
        # Load audio file
        speech_array, sampling_rate = librosa.load(file_path, sr=target_sr)
        
        # Return the speech array
        return speech_array
    
    except Exception as e:
        print(f"Error loading {file_path}: {str(e)}")
        return None

# Create a DataFrame with file paths and labels
def create_dataframe(file_list):
    data = []
    for file_path in file_list:
        # Extract label from filename (e.g., 'file_id_emotion.wav')
        label = file_path.split('_')[-2].replace('.wav', '')
        data.append({'file_path': file_path, 'label': label})
    return pd.DataFrame(data)

In [14]:
# Create dataframes for training, validation, and test sets
train_df = create_dataframe(train_files)
val_df = create_dataframe(val_files)
test_df = create_dataframe(test_files)

# Create datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [15]:
print(train_dataset)
print(val_dataset)
print(test_dataset)

Dataset({
    features: ['file_path', 'label'],
    num_rows: 2478
})
Dataset({
    features: ['file_path', 'label'],
    num_rows: 531
})
Dataset({
    features: ['file_path', 'label'],
    num_rows: 532
})


# Preprocess the Datasets

In [16]:
label2id = {'hap': 0, 'sad': 1, 'ang': 2, 'neu': 3}

processor = AutoProcessor.from_pretrained("elgeish/wav2vec2-large-xlsr-53-arabic")

def preprocess_function(examples):
    audio_list = []
    max_length = 0
    
    for file_path in examples['file_path']:
        # Load the audio file
        waveform, sample_rate = librosa.load(file_path, sr=16000)
        
        # Update the maximum length
        if len(waveform) > max_length:
            max_length = len(waveform)
        
        audio_list.append(waveform)
    
    # Pad/truncate all audio to the maximum length
    padded_audio_list = []
    for waveform in audio_list:
        if len(waveform) > max_length:
            waveform = waveform[:max_length]
        else:
            pad_length = max_length - len(waveform)
            waveform = np.pad(waveform, (0, pad_length), 'constant')
        padded_audio_list.append(waveform)
    
    # Convert padded audio list to PyTorch tensor
    inputs = processor(padded_audio_list, sampling_rate=16000, return_tensors="pt", padding=True)
    
    # Convert labels to numerical IDs
    labels = [label2id[label] for label in examples['label']]
    
    # Ensure labels are tensors
    inputs['labels'] = torch.tensor(labels, dtype=torch.long)
    
    return inputs

# Apply the preprocess function to the datasets
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['file_path'])
val_dataset = val_dataset.map(preprocess_function, batched=True, remove_columns=['file_path'])
test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=['file_path'])


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/2478 [00:00<?, ? examples/s]

Map:   0%|          | 0/531 [00:00<?, ? examples/s]

Map:   0%|          | 0/532 [00:00<?, ? examples/s]

In [17]:
# Verify labels are integers
print(type(train_dataset[0]['labels']))  # Should print <class 'int'>
print(type(val_dataset[0]['labels']))    # Should print <class 'int'>
print(type(test_dataset[0]['labels']))   # Should print <class 'int'>

# Verify audio inputs are tensors
print(type(train_dataset[0]['input_values']))  # Should print <class 'torch.Tensor'>
print(type(val_dataset[0]['input_values']))    # Should print <class 'torch.Tensor'>
print(type(test_dataset[0]['input_values']))   # Should print <class 'torch.Tensor'>

<class 'int'>
<class 'int'>
<class 'int'>
<class 'list'>
<class 'list'>
<class 'list'>


# Solve Data imbalance problem

In [18]:
# Define class weights based on the provided counts
class_counts = {"ang": 741, "hap": 392, "neu": 808, "sad": 537}
total_samples = sum(class_counts.values())
class_weights = {cls: total_samples / count for cls, count in class_counts.items()}
class_weights_tensor = torch.tensor([class_weights["neu"], class_weights["ang"], class_weights["sad"], class_weights["hap"]], dtype=torch.float)
print(class_weights_tensor)


tensor([3.0668, 3.3441, 4.6145, 6.3214])


In [19]:
# Custom Trainer to use class weights
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # Use class weights
        class_weights = torch.tensor(self.class_weights).to(self.args.device)
        loss = F.cross_entropy(logits, labels, weight=class_weights)
        
        return (loss, outputs) if return_outputs else loss

# Define Accuracy Metric

In [20]:
accuracy_metric = load_metric("accuracy")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

/tmp/ipykernel_35/3547444929.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


# Saving Checkpoint

In [23]:
class CustomSaveCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        try:
            # Define the output directory in Google Drive
            output_dir = '/content/drive/MyDrive/Wav2Vec2_Checkpoints'
            model = kwargs['model']
            optimizer = kwargs['optimizer']
            
            # Save model checkpoint
            model_path = os.path.join(output_dir, "pytorch_model.bin")
            optimizer_path = os.path.join(output_dir, "optimizer.pt")

            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
                
            torch.save(model.state_dict(), model_path)
            torch.save(optimizer.state_dict(), optimizer_path)
            
            print(f"Checkpoint saved successfully at {output_dir}")
        except Exception as e:
            print(f"Error saving checkpoint: {e}")

# Custom Class to visualize learning curve

In [26]:
# Logging callback to record metrics
class LoggingCallback(TrainerCallback):
    def __init__(self):
        self.train_loss = []
        self.eval_loss = []
        self.train_accuracy = []
        self.eval_accuracy = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            self.train_loss.append(logs.get("loss"))
            self.eval_loss.append(logs.get("eval_loss"))
            self.train_accuracy.append(logs.get("train_accuracy"))
            self.eval_accuracy.append(logs.get("eval_accuracy"))

    def plot_metrics(self):
        epochs = range(1, len(self.train_loss) + 1)
        
        plt.figure(figsize=(14, 7))
        plt.subplot(1, 2, 1)
        plt.plot(epochs, self.train_loss, label="Training Loss")
        plt.plot(epochs, self.eval_loss, label="Validation Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.title("Training and Validation Loss")
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(epochs, self.train_accuracy, label="Training Accuracy")
        plt.plot(epochs, self.eval_accuracy, label="Validation Accuracy")
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.title("Training and Validation Accuracy")
        plt.legend()
        
        plt.show()

# Load the wavtovec pretrained model

In [27]:
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    "elgeish/wav2vec2-large-xlsr-53-arabic",
    num_labels=len(label2id),
    ignore_mismatched_sizes=True,
    problem_type="single_label_classification",
)

# Initialize the logging callback
logging_callback = LoggingCallback()

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at elgeish/wav2vec2-large-xlsr-53-arabic and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Define training arguments and trainer

In [28]:
# Define your training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Wav2Vec2_Checkpoints",               # Directory to save checkpoints
    weight_decay=0.01,
    learning_rate = 2e-5,
    num_train_epochs= 15,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir="/content/drive/MyDrive/Wav2Vec2_Logs",                 # Directory for logging
    logging_steps=10,
    eval_strategy="epoch",          # Evaluate at the end of each epoch
    save_strategy="epoch",                # Save checkpoint at the end of each epoch
    save_total_limit=2,                   # Limit the total number of checkpoints
    load_best_model_at_end=True,          # Load the best model at the end
    metric_for_best_model="accuracy",     # Metric to use to select the best model
    greater_is_better=True,               # Whether a greater metric value is better
    fp16=True,
    run_name = "Egyptain Arabic SER"
)

# Define the data collator
def custom_data_collator(features):
    batch = {k: [f[k] for f in features] for k in features[0].keys()}
    labels = torch.tensor(batch.pop('labels'), dtype=torch.long)
    inputs = processor(batch['input_values'], sampling_rate=16000, return_tensors="pt", padding=True)
    inputs['labels'] = labels
    return inputs

# Initialize the Trainer
trainer =CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=custom_data_collator,
    tokenizer=processor.feature_extractor,  # or processor if feature_extractor is not available
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3), logging_callback],
    class_weights=class_weights_tensor.tolist(),  # Add class weights to training arguments
)


# Train Model

In [29]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,1.247500,0.940551,0.621469
2,0.662700,0.831629,0.640301
3,0.760000,0.734657,0.702448
4,0.734800,0.768250,0.694915
5,0.652800,0.832416,0.696798
6,0.279600,1.083706,0.715631
7,0.162200,1.580787,0.708098
8,0.893600,2.878526,0.719397
9,0.289500,4.076486,0.728814
10,0.748900,4.408115,0.721281


TrainOutput(global_step=4340, training_loss=0.4688099199411175, metrics={'train_runtime': 14787.5746, 'train_samples_per_second': 2.514, 'train_steps_per_second': 0.314, 'total_flos': 1.2168970815461659e+19, 'train_loss': 0.4688099199411175, 'epoch': 14.0})

# Save best Model

In [1]:
# Save the best model
torch.save(model.state_dict(), '/kaggle/working/best_model.pth')

NameError: name 'torch' is not defined

# Plot Learning Curve

In [ ]:
# Plot the metrics after training
logging_callback.plot_metrics()

# Evaluate Model

In [ ]:
# Evaluate on the validation dataset
val_results = trainer.evaluate(eval_dataset=val_dataset)
val_accuracy = val_results['eval_accuracy']
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Evaluate on the test dataset
test_results = trainer.predict(test_dataset)
test_accuracy = test_results.metrics['test_accuracy']
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predict on the test dataset
predictions = trainer.predict(test_dataset).predictions
predicted_labels = predictions.argmax(axis=1)
true_labels = test_dataset["labels"]

# Compute the classification report
report = classification_report(true_labels, predicted_labels, target_names=["neu", "ang", "sad", "hap"])

# Display the classification report
print("Classification Report:")
print(report)

# Display Confusion matrix

In [32]:
# Define your emotions based on your label mapping
emotions = ['happy', 'sad', 'angry', 'neutral']

# Function to plot confusion matrix
def plot_confusion_matrix(conf_matrix, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=emotions, yticklabels=emotions)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(title)
    plt.show()


# Assuming your predictions are in `test_results.predictions`
test_predictions = np.argmax(test_results.predictions, axis=1)
test_labels = test_dataset["labels"]

# Calculate confusion matrix
test_conf_matrix = confusion_matrix(test_labels, test_predictions)

# Plot Test Confusion Matrix
print("Test Confusion Matrix:")
plot_confusion_matrix(test_conf_matrix, title='Test Confusion Matrix')

NameError: name 'test_results' is not defined

In [ ]:
"""# Function to compute emotion percentages
def compute_emotion_percentages(model, tokenizer, audio_file_path):
    inputs = tokenizer(audio_file_path, return_tensors="pt", sampling_rate=16000, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy().flatten()
        percentages = {emotion: round(prob * 100, 2) for emotion, prob in zip(["neutral", "angry", "sad", "happy"], probabilities)}
    return percentages

# Example usage
audio_file_path = "path_to_audio_file.wav"  # Replace with your audio file path
emotion_percentages = compute_emotion_percentages(model, tokenizer, audio_file_path)
print(emotion_percentages)"""